In [1]:
import os
import numpy as np
from scipy.interpolate import griddata
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
# Répertoires fichiers grilles
rep = '/home/thibault-delahaye/git_dev/notebook-grid-tools/DATASETS_CROCOTOOLS/Topo/to_merge_ter/'

# Chemins des rasters à fusionner
input1 = rep + "MNT_NC100m_TSUCAL_GEO_refNM_ZNEG_V1.0.grd" # HIGH RESOLUTION
#input2 = rep + "gebco_2024_n-14.502_s-25.4004_w158.5986_e171.6064.nc" #LOW RESOLUTION
input2= rep + "etopo_2022_NC.tiff"
#OUTPUT de la fusion
#output = rep + "gebco_x_tsucal"

In [5]:
# Ouvrir le fichier 1
ds1 = xr.open_dataset(input1)

# Ouvrir le fichier 2
#try:
#    import rioxarray
#    ds2_dataArray= rioxarray.open_rasterio(input2)
    # Transformer le DataArray en Dataset
#    ds2_temp = ds2_dataArray.to_dataset()
#    ds2= ds2_temp.sel(band=1, drop=True)
#except:
ds2 = xr.open_dataset(input2)

In [ ]:
if input2[-4:]=='tiff':
    ds2= ds2.sel(band=1, drop=True)

In [17]:
# Identifier la variable contenant les données de bathymétrie
# On suppose que la variable de bathymétrie est la seule avec des valeurs numériques

for var_name in ds1.data_vars:
    var_data = ds1[var_name]
    # Vérifier si la variable est 2D et contient des valeurs numériques
    if var_data.ndim == 2 and var_data.dtype.kind in {'f', 'i'}:
         z_ds1= var_name
         break
else:
    print("Aucune variable de bathymétrie trouvée dans ds1")

for var_name in ds2.data_vars:
    var_data = ds2[var_name]
    # Vérifier si la variable est 2D et contient des valeurs numériques
    if var_data.ndim == 2 and var_data.dtype.kind in {'f', 'i'}:
         z_ds2= var_name
         break
else:
    print("Aucune variable de bathymétrie trouvée dans ds2")

In [18]:
# Identifier les noms des coordonnées lat/lon des fichiers

for coord_name in ds1.coords:
    coord_data = ds1[coord_name]
    # On suppose que lon et lat sont généralement des coordonnées avec des valeurs géographiques
    if coord_name.lower() in {'lon', 'longitude'}:
        lon_coord_ds1  = coord_name
    elif coord_name.lower() in {'lat', 'latitude'}:
        lat_coord_ds1  = coord_name
    elif coord_data.dims == ('x',) or coord_data.dims == ('y',):
        if 'x' in coord_data.dims:
            lon_coord_ds1 = coord_name
        if 'y' in coord_data.dims:
            lat_coord_ds1 = coord_name

for coord_name in ds2.coords:
    coord_data = ds2[coord_name]
    # On suppose que lon et lat sont généralement des coordonnées avec des valeurs géographiques
    if coord_name.lower() in {'lon', 'longitude'}:
        lon_coord_ds2  = coord_name
    elif coord_name.lower() in {'lat', 'latitude'}:
        lat_coord_ds2  = coord_name
    elif coord_data.dims == ('x',) or coord_data.dims == ('y',):
        if 'x' in coord_data.dims:
            lon_coord_ds2 = coord_name
        if 'y' in coord_data.dims:
            lat_coord_ds2 = coord_name

In [19]:
#Interpoler la grille de basse résolution sur une grille de résolution = HIGH RES

#Déterminer la résolution de la grille 1
resolution_lon= np.diff(ds1[lon_coord_ds1]).mean()
resolution_lat= np.diff(ds1[lat_coord_ds1]).mean()

lon2= ds2[lon_coord_ds2].values
lat2= ds2[lat_coord_ds2].values
z2= ds2[z_ds2].values

# Récupérér grille ds2 initiale
lon_grid_2, lat_grid_2 = np.meshgrid(lon2, lat2)

In [20]:
# Aplatir les grilles 2D et les valeurs
lon_flat_2 = lon_grid_2.ravel()
lat_flat_2 = lat_grid_2.ravel()
z_flat_2 = z2.ravel()

# Créer une nouvelle grille ds2
new_lon_2 = np.arange(lon2.min(), lon2.max(), resolution_lon)
new_lat_2 = np.arange(lat2.min(), lat2.max(), resolution_lat)
new_lon_grid_2, new_lat_grid_2 = np.meshgrid(new_lon_2, new_lat_2)

In [21]:
z2_interp = griddata((lon_flat_2, lat_flat_2), z_flat_2, (new_lon_grid_2, new_lat_grid_2), method='nearest')

In [22]:
lon1= ds1[lon_coord_ds1].values
lat1= ds1[lat_coord_ds1].values
z1= ds1[z_ds1].values

# Récupérér grille ds2 initiale
lon_grid_1, lat_grid_1 = np.meshgrid(lon1, lat1)

# Aplatir les grilles 2D et les valeurs
lon_flat_1 = lon_grid_1.ravel()
lat_flat_1 = lat_grid_1.ravel()
z_flat_1 = z1.ravel()

# 1. Identifier la zone de chevauchement entre les grilles
lon_min_1, lon_max_1 = lon1.min(), lon1.max()  # Étendue de la grille 1 en longitude
lat_min_1, lat_max_1 = lat1.min(), lat1.max()  # Étendue de la grille 1 en latitude

# Créer un masque sur la grille rééchantillonnée de la grille 2 qui couvre seulement la zone de la grille 1
mask_overlap = (new_lon_grid_2 >= lon_min_1) & (new_lon_grid_2 <= lon_max_1) & \
               (new_lat_grid_2 >= lat_min_1) & (new_lat_grid_2 <= lat_max_1)

# 2. Interpolation des données de la grille 1 (haute résolution) uniquement sur la zone couverte par la grille 1
z1_interp_on_z2_overlap = griddata(
    (lon_flat_1, lat_flat_1),  # Points d'origine (grille 1)
    z_flat_1,                  # Valeurs d'origine (grille 1)
    (new_lon_grid_2[mask_overlap], new_lat_grid_2[mask_overlap]),  # Points cibles (zone de la grille 1 dans la grille 2)
    method='nearest'           # Méthode d'interpolation
)


In [23]:
z2_save= z2_interp.copy()

# 3. Remplacer les données de z2_interp uniquement dans la zone de chevauchement avec celles de la grille 1
z2_interp[mask_overlap] = z1_interp_on_z2_overlap

### LISSAGE

In [24]:
# Définir la largeur de la zone tampon en nombre de mailles
buffer_width = 30


#Bords de la grille haute résolution dans la grille finale
lon_min= new_lon_grid_2[mask_overlap].min()
lon_max= new_lon_grid_2[mask_overlap].max()
lat_min= new_lat_grid_2[mask_overlap].min()
lat_max= new_lat_grid_2[mask_overlap].max()

# Calculer les limites extérieures de la zone tampon
lon_buffer_min = lon_min - buffer_width * resolution_lon
lon_buffer_max = lon_max + buffer_width * resolution_lon
lat_buffer_min = lat_min - buffer_width * resolution_lat
lat_buffer_max = lat_max + buffer_width * resolution_lat

#Masque: zone tampon + grille haute résolution
mask_buff_extremum=(new_lon_grid_2 >= lon_buffer_min) & (new_lon_grid_2 <= lon_buffer_max) & \
                   (new_lat_grid_2 >= lat_buffer_min) & (new_lat_grid_2 <= lat_buffer_max)

# Zone tampon excluant la grille haute résolution
mask_buffer= mask_buff_extremum & ~mask_overlap

### Lissage des nan

In [25]:
mask_nan = np.isnan(z2_interp) & ~np.isnan(z2_save)
z2_combined= z2_interp.copy()
z2_combined[mask_nan]= z2_save[mask_nan]

In [26]:
mask_nan.shape

(7734, 12459)

In [27]:
from scipy.interpolate import RectBivariateSpline
# Créer un interpolateur avec les nouvelles coordonnées
spline = RectBivariateSpline(new_lat_grid_2[:, 0], new_lon_grid_2[0, :], z2_combined)

# Interpoler sur les nouvelles coordonnées
z2_smoothed = spline(new_lat_grid_2[:, 0], new_lon_grid_2[0, :])

###
#z2_interp[mask_nan]= z2_smoothed[mask_nan]

In [28]:
z2_interp[mask_nan]= z2_smoothed[mask_nan]

In [16]:
# Masque de la limite extérieure de la zone tampon (partie grille 2)
#lon_buffer_min_ext = lon_buffer_min - 30*resolution_lon
#lon_buffer_max_ext = lon_buffer_max + 30*resolution_lon
#lat_buffer_min_ext = lat_buffer_min - 30*resolution_lat
#lat_buffer_max_ext = lat_buffer_max + 30*resolution_lat

#Masque: zone tampon + grille haute résolution
#mask_buff_ext_extremum=(new_lon_grid_2 >= lon_buffer_min_ext) & (new_lon_grid_2 <= lon_buffer_max_ext) & \
#                   (new_lat_grid_2 >= lat_buffer_min_ext) & (new_lat_grid_2 <= lat_buffer_max_ext)

#mask_buff_ext= mask_buff_ext_extremum & ~mask_buff_extremum

IndentationError: unexpected indent (1040564234.py, line 9)

In [29]:
import numpy as np
from scipy.ndimage import binary_dilation

# Appliquer une dilatation binaire pour obtenir les voisins externes
dilated_mask_buffer_ext = binary_dilation(mask_buff_extremum)

# Les bords externes sont ceux qui sont dans la dilatation mais pas dans le masque initial
border_buffer_ext = dilated_mask_buffer_ext & ~mask_buff_extremum

# Récupérer les indices des bords extérieurs
#indices_border_buff_ext = np.where(border_buffer_ext)

In [18]:
# Masque de la limite intérieure de la zone tampon (partie grille 1)
#lon_buffer_min_int = lon_min + 3*resolution_lon
#lon_buffer_max_int = lon_max - 3*resolution_lon
#lat_buffer_min_int = lat_min + 3*resolution_lat
#lat_buffer_max_int = lat_max - 3*resolution_lat

#Masque: zone tampon + grille haute résolution
#mask_buff_int_extremum=(new_lon_grid_2 >= lon_buffer_min_int) & (new_lon_grid_2 <= lon_buffer_max_int) & \
  #                 (new_lat_grid_2 >= lat_buffer_min_int) & (new_lat_grid_2 <= lat_buffer_max_int)

#mask_buff_int= ~mask_buff_int_extremum & mask_overlap

In [30]:
from scipy.ndimage import binary_erosion
#Appliquer une érosion binaire pour identifier l'intérieur
eroded_mask = binary_erosion(mask_overlap)

# Les bords intérieurs sont ceux qui sont dans le masque initial mais pas dans l'érosion
border_mask_inner = mask_overlap & ~eroded_mask


# Récupérer les indices des bords intérieurs
#indices_border_inner = np.where(border_mask_inner)

In [20]:
# Coordonnées et valeurs pour la zone tampon
#lon_buffer = new_lon_grid_2[mask_buffer]
#lat_buffer = new_lat_grid_2[mask_buffer]

# Coordonnées et valeurs pour la grille 1 utiles
#lon_buffer_int = new_lon_grid_2[mask_buff_int]
#lat_buffer_int = new_lat_grid_2[mask_buff_int]

# Coordonnées et valeurs pour la grille 2 utiles
#lon_buffer_ext = new_lon_grid_2[mask_buff_ext]
#lat_buffer_ext = new_lat_grid_2[mask_buff_ext]

In [21]:
# Coordonnées des points dans la zone tampon
#coords_buffer = np.vstack((lon_buffer, lat_buffer)).T

# Coordonnées des points dans la grille 1 utiles
#coords_buffer_int = np.vstack((lon_buffer_int, lat_buffer_int)).T

# Coordonnées des points dans la grille 1 utiles
#coords_buffer_ext = np.vstack((lon_buffer_ext, lat_buffer_ext)).T

In [22]:
#from scipy.spatial import cKDTree

# Coordonnées des points de la grille haute résolution
#coords_high_res = np.vstack((new_lon_grid_2[mask_overlap].ravel(), new_lat_grid_2[mask_overlap].ravel())).T

# Créer un arbre KD pour les points de la grille haute résolution
#tree_high_res = cKDTree(coords_buffer_int)

In [23]:
# Créer un arbre KD pour les points de la grille basse résolution
#tree_low_res = cKDTree(coords_buffer_ext)

In [24]:
# Trouver les indices des points les plus proches et les distances
#distances_to_high_res, indices_high_res = tree_high_res.query(coords_buffer)
# Trouver les distances aux points les plus proches de la grille 2 (hors de la zone tampon)
#distances_to_low_res, indices_low_res = tree_low_res.query(coords_buffer)

In [31]:
# Rayon approximatif de la Terre en mètres
#R = 6371000  # Rayon moyen de la Terre en mètres

# Fonction pour calculer la distance de Haversine
#def haversine(lon1, lat1, lon2, lat2):
#    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
#    dlon = lon2 - lon1
#    dlat = lat2 - lat1
#    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
#    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
#    return R * c

lon_inner = new_lon_grid_2[border_mask_inner]
lat_inner = new_lat_grid_2[border_mask_inner]
lon_outer = new_lon_grid_2[border_buffer_ext]
lat_outer = new_lat_grid_2[border_buffer_ext]

# Extraire les valeurs de z2_interp aux bords intérieurs et extérieurs
values_inner = z2_interp[border_mask_inner]
values_outer = z2_interp[border_buffer_ext]

# Pour chaque maille dans mask_buffer, trouver les valeurs à partir des bords intérieurs et extérieurs les plus proches
buffer_lon = new_lon_grid_2[mask_buffer]
buffer_lat = new_lat_grid_2[mask_buffer]

# Initialiser les tableaux pour stocker les valeurs interpolées
interpolated_values = np.zeros(buffer_lon.shape)

# Pour chaque maille dans mask_buffer
for i, (lon, lat) in enumerate(zip(buffer_lon, buffer_lat)):
    # Calculer les distances à toutes les mailles de border_mask_inner et border_mask_ext
    #dist_inner = haversine(lon, lat, lon_inner, lat_inner)
    #dist_outer = haversine(lon, lat, lon_outer, lat_outer)
    dist_inner = np.sqrt((lon_inner - lon)**2 + (lat_inner - lat)**2)
    dist_outer = np.sqrt((lon_outer - lon)**2 + (lat_outer - lat)**2)
    
    
    # Trouver les indices des bords les plus proches
    nearest_inner_idx = np.argmin(dist_inner)
    nearest_outer_idx = np.argmin(dist_outer)
    
    # Extraire les valeurs les plus proches
    value_inner = values_inner[nearest_inner_idx]
    value_outer = values_outer[nearest_outer_idx]
    
    # Calculer la distance totale pour la pondération
    total_distance = dist_inner[nearest_inner_idx] + dist_outer[nearest_outer_idx]
    
    # Calculer les poids pour l'interpolation
    weight_inner = dist_outer[nearest_outer_idx] / total_distance
    weight_outer = dist_inner[nearest_inner_idx] / total_distance
    
    # Calculer la valeur interpolée
    interpolated_values[i] = weight_inner * value_inner + weight_outer * value_outer


z2_interp[mask_buffer] = interpolated_values

In [28]:
# Éviter les divisions par zéro (si un point est très proche d'un autre point)
#epsilon = 1e-10
#distances_to_high_res = distances_to_high_res + epsilon
#distances_to_low_res = distances_to_low_res + epsilon

# Calculer les pondérations inverses aux distances
#weights_high_res = 1 / distances_to_high_res
#weights_low_res = 1 / distances_to_low_res

# Normaliser les pondérations pour que leur somme soit égale à 1
#weights_sum = weights_high_res + weights_low_res
#weights_high_res /= weights_sum
#weights_low_res /= weights_sum

#weights_high_res = 1 - (distances_to_high_res / (distances_to_high_res + distances_to_low_res))
#weights_low_res = 1 - weights_high_res

In [29]:
# Extraire les valeurs interpolées des deux grilles
#z_high_res_nearest = z2_interp[mask_buff_int][indices_high_res]
#z_low_res_nearest = z2_interp[mask_buff_ext][indices_low_res]

In [30]:
# Calculer la valeur lissée pour chaque point dans la zone tampon
#z_smooth = weights_high_res * z_high_res_nearest + weights_low_res * z_low_res_nearest
#z_smooth = (weights_high_res * z_high_res_nearest + weights_low_res * z_low_res_nearest + z2_interp[mask_buffer]) / 2


In [ ]:
# Remplacer les valeurs dans la zone tampon
#z2_interp[mask_buffer] = z_smooth

In [ ]:
#import scipy.ndimage as ndimage
#mask_nan = np.isnan(z2_interp)
#z2_interp[mask_nan] = z2_save[mask_nan]
# Appliquer un filtre de lissage (gaussien) sur la zone des trous comblés
# sigma contrôle l'intensité du lissage (ajuster en fonction des résultats)
#z_smooth_nan = ndimage.gaussian_filter(z2_save, sigma=3)
#z2_interp[mask_nan] = z_smooth_nan[mask_nan]

In [32]:
import xarray as xr

# Obtenez les coordonnées uniques
new_lon_unique = np.unique(new_lon_grid_2)
new_lat_unique = np.unique(new_lat_grid_2)

# Créer un DataArray pour les données interpolées
ds_interpolated = xr.DataArray(
    z2_interp, 
    coords=[('lat', new_lat_unique), ('lon', new_lon_unique)],  # Ajouter les coordonnées
    dims=['lat', 'lon']  # Spécifier les dimensions
)

# Créer un Dataset pour organiser les variables
ds_to_save = xr.Dataset({
    'z2_interp': ds_interpolated  # Ajouter la variable interpolée
})

# Sauvegarder dans un fichier NetCDF
ds_to_save.to_netcdf('composite_etopo_jroger.nc')


In [33]:
ds2.to_netcdf('etopo.nc')